In [1]:
#imports
import os
import glob as gb
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
import tempfile

#tensorflow imports
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#import tfmot and numpy
!pip install tensorflow-model-optimization
import tensorflow_model_optimization as tfmot
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#mount to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#copy dataset from drive
!cp -r ./drive/MyDrive/fer.zip ./

#import
import zipfile

with zipfile.ZipFile("./fer.zip", "r") as zip_ref:
    zip_ref.extractall("./")

#copy validation 
!cp -r "./test" "./validation" 

#paths to sets
train_dir = "./train"
test_dir = "./test"
val_dir = "./validation"

#remove Public images from testing dir
for root, subdirs, files in os.walk(test_dir):
    for file in files:
        if file.startswith("Public"):
            os.remove(os.path.join(root,file))



#remove Private images from validation dir
for root, subdirs, files in os.walk(val_dir):
    for file in files:
        if file.startswith("Private"):
            os.remove(os.path.join(root,file))

#set batch size and classes
batch = 64
classes = ["angry", "fear", "happy","neutral","sad","surprise"]

In [5]:
#set training data without data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255)

#normalize validation data
val_datagen = ImageDataGenerator(rescale = 1./255)

#normalize test data
test_datagen = ImageDataGenerator(rescale = 1./255)



#get training set
training_set = train_datagen.flow_from_directory(train_dir,\
                                                 target_size = (48, 48),\
                                                 color_mode = "grayscale",\
                                                 class_mode = "categorical",\
                                                 shuffle = True,\
                                                 batch_size=batch,\
                                                 classes=classes)

#get val set
val_set = val_datagen.flow_from_directory(val_dir, target_size = (48, 48),\
                                            color_mode = "grayscale",\
                                            class_mode = "categorical",\
                                            classes = classes,\
                                            batch_size=batch,\
                                            shuffle = False)

#get test set
test_set = test_datagen.flow_from_directory(test_dir, target_size = (48, 48),\
                                            color_mode = "grayscale",\
                                            class_mode = "categorical",\
                                            classes = classes,\
                                            batch_size=batch,\
                                            shuffle = False)
print(training_set.class_indices)
print("angry", training_set.classes[training_set.classes == 0].shape[0])
print("fear", training_set.classes[training_set.classes == 1].shape[0])
print("happy", training_set.classes[training_set.classes == 2].shape[0])
print("neutral", training_set.classes[training_set.classes == 3].shape[0])
print("sad", training_set.classes[training_set.classes == 4].shape[0])
print("surprise", training_set.classes[training_set.classes == 5].shape[0])

Found 28273 images belonging to 6 classes.
Found 3533 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.
{'angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4, 'surprise': 5}
angry 3995
fear 4097
happy 7215
neutral 4965
sad 4830
surprise 3171


In [6]:
#load model for pruning
model_path = "./drive/MyDrive/model1_100epoch.h5"
model1 = load_model(model_path)

In [7]:
#get baseline accuracy
_, baseline_model_accuracy = model1.evaluate(
    test_set, verbose=0)
print("Baseline accuracy: ", baseline_model_accuracy)

Baseline accuracy:  0.6986417770385742


In [8]:
#todo optimize with pruning (and quantization) https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch = 64
epochs = 2

num_images = training_set.n
end_step = np.ceil(num_images / batch).astype(np.int32) * epochs // 2

# Define model for pruning.
pruning_params = {
      "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.40,
                                                               final_sparsity=0.55,
                                                               begin_step=0,
                                                               end_step=end_step)
}


model_for_pruning = prune_low_magnitude(model1, **pruning_params)

model_for_pruning.compile(optimizer='adam',
                          loss=tf.keras.losses.CategoricalCrossentropy(),
                          metrics=['accuracy'])

#model_for_pruning.summary()


In [9]:
#fit 
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(training_set,
                  batch_size=batch, epochs=epochs, validation_data = val_set,
                  callbacks=callbacks)

Epoch 1/2
  6/442 [..............................] - ETA: 18s - loss: 0.4200 - accuracy: 0.8490

442/442 [==============================] - 39s 55ms/step - loss: 0.5822 - accuracy: 0.7877 - val_loss: 0.9511 - val_accuracy: 0.6637
Epoch 2/2
442/442 [==============================] - 21s 47ms/step - loss: 0.5407 - accuracy: 0.8015 - val_loss: 0.9575 - val_accuracy: 0.6841


In [10]:
#Compare to baseline
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_set, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.6986417770385742
Pruned test accuracy: 0.6949632167816162


In [11]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

model_for_export.compile(optimizer='adam',
                          loss=tf.keras.losses.CategoricalCrossentropy(),
                          metrics=['accuracy'])

tf.keras.models.save_model(model_for_export, "./drive/MyDrive/fer2013_pruned.h5", include_optimizer=False)

In [12]:
test_model = load_model("./drive/MyDrive/fer2013_pruned.h5")

test_model.compile(optimizer='adam',
                          loss=tf.keras.losses.CategoricalCrossentropy(),
                          metrics=['accuracy'])

test_model.evaluate(test_set)

56/56 [==============================] - 2s 31ms/step - loss: 0.9271 - accuracy: 0.6950


[0.9271339774131775, 0.6949632167816162]